In [1]:
import findspark
findspark.init('/home/pradyut/spark-3.2.3-bin-hadoop3.2-scala2.13')

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession


In [4]:
sc = SparkSession.builder.appName("App1").getOrCreate()


23/01/13 08:48:31 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.78.128 instead (on interface ens33)
23/01/13 08:48:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/13 08:48:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = sc.read.json("people.json")

In [6]:
df.show(2)

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
+----+-------+
only showing top 2 rows



In [7]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [8]:
df.columns

['age', 'name']

In [13]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [14]:
#Describing Schemas 
from pyspark.sql.types import StructField,IntegerType,StringType,StructType

In [15]:
data_schema = [StructField('age',IntegerType(),True),StructField('name',StringType(),True)]

In [16]:
final_struc = StructType(fields=data_schema)

In [17]:
df2 = sc.read.json('people.json',schema=final_struc)
df2.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [18]:
df2.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [19]:
#Working with dataframes
type(df['age'])

pyspark.sql.column.Column

In [20]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [23]:
print(df.head(2))
print(df.head(2)[0])

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]
Row(age=None, name='Michael')


In [25]:
df.withColumn('New_age',df['age']+3).show()

+----+-------+-------+
| age|   name|New_age|
+----+-------+-------+
|null|Michael|   null|
|  30|   Andy|     33|
|  19| Justin|     22|
+----+-------+-------+



In [26]:
df.withColumnRenamed('age','new_col_age').show()

+-----------+-------+
|new_col_age|   name|
+-----------+-------+
|       null|Michael|
|         30|   Andy|
|         19| Justin|
+-----------+-------+



In [27]:
#Using SQL to work with.
df.createOrReplaceTempView('People')

In [29]:
res = sc.sql('select * from People where age > 20')

In [30]:
res.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



#Spark Dataframe Basic Operations

In [31]:
sc1 = SparkSession.builder.appName('Basic_Ops1').getOrCreate()

In [33]:
df1 = sc1.read.csv('appl_stock.csv',inferSchema=True,header=True)

In [34]:
df1.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [37]:
df1.show(3)

+----------+----------+----------+------------------+----------+---------+------------------+
|      Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|
+----------+----------+----------+------------------+----------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|
|2010-01-06|214.379993|    215.23|        210.750004|210.969995|138040000|27.333178000000004|
+----------+----------+----------+------------------+----------+---------+------------------+
only showing top 3 rows



In [38]:
df1.head(1)

[Row(Date='2010-01-04', Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)]

In [40]:
df1.filter('close < 500').select('open').show()

+------------------+
|              open|
+------------------+
|        213.429998|
|        214.599998|
|        214.379993|
|            211.75|
|        210.299994|
|212.79999700000002|
|209.18999499999998|
|        207.870005|
|210.11000299999998|
|210.92999500000002|
|        208.330002|
|        214.910006|
|        212.079994|
|206.78000600000001|
|202.51000200000001|
|205.95000100000001|
|        206.849995|
|        204.930004|
|        201.079996|
|192.36999699999998|
+------------------+
only showing top 20 rows



In [44]:
df1.filter(df1['close']<500).select(['open','close']).show()

+------------------+------------------+
|              open|             close|
+------------------+------------------+
|        213.429998|        214.009998|
|        214.599998|        214.379993|
|        214.379993|        210.969995|
|            211.75|            210.58|
|        210.299994|211.98000499999998|
|212.79999700000002|210.11000299999998|
|209.18999499999998|        207.720001|
|        207.870005|        210.650002|
|210.11000299999998|            209.43|
|210.92999500000002|            205.93|
|        208.330002|        215.039995|
|        214.910006|            211.73|
|        212.079994|        208.069996|
|206.78000600000001|            197.75|
|202.51000200000001|        203.070002|
|205.95000100000001|        205.940001|
|        206.849995|        207.880005|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
+------------------+------------------+
only showing top 20 rows



In [46]:
df1.filter((df1['close'] < 200) & (df1['open']>200)).show()

+----------+------------------+----------+----------+----------+---------+------------------+
|      Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+



In [47]:
df1.filter((df1['close'] < 200) & ~(df1['open']>200)).show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-02-01|192.36999699999998|             196.0|191.29999899999999|        194.729998|187469100|         25.229131|
|2010-02-02|        195.909998|        196.319994|193.37999299999998|        195.859997|174585600|25.375532999999997|
|2010-02-03|        195.169994|        200.200003|        194.420004|        199.229994|153832000|25.812148999999998|
|2010-02-04|        196.730003|        198.370001|        191.570005|        192.050003|189413000|         24.881912|
|2010-02-05|192.63000300000002|             196.0|        190.850002|        195.460001|212576700|25.323710000000002|
|2010-02-08|        195.690006|197.88000300000002|      

In [48]:
#Collecting result as Rows - As list
rows = df1.filter(df1['Low']>100).collect()

In [50]:
#To get a particular Row
row = rows[1]

In [51]:
row.asDict()

{'Date': '2010-01-05',
 'Open': 214.599998,
 'High': 215.589994,
 'Low': 213.249994,
 'Close': 214.379993,
 'Volume': 150476200,
 'Adj Close': 27.774976000000002}

## Group By and Aggregate functions

In [53]:
sc3 = SparkSession.builder.appName('Basic_App2').getOrCreate()

In [54]:
df3 = sc3.read.csv('sales_info.csv',inferSchema=True,header=True)

In [55]:
df3.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [56]:
df3.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [57]:
df3.groupBy('Company')

In [59]:
df3.groupBy('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [61]:
df3.agg({'Sales':'max'}).show() #Aggregate function takes args as dict

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



In [63]:
df3.groupBy('Company').agg({'Sales':'count'}).withColumnRenamed('count(Sales)','Sales_Count').show()

+-------+-----------+
|Company|Sales_Count|
+-------+-----------+
|   APPL|          4|
|   GOOG|          3|
|     FB|          2|
|   MSFT|          3|
+-------+-----------+



In [66]:
from pyspark.sql.functions import *

In [69]:
df3.select(avg('Sales').alias('Avg_Sales')).show()

+-----------------+
|        Avg_Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [72]:
sales_std = df3.select(stddev('Sales').alias('std'))

In [74]:
sales_std.select(format_number('std',2).alias('std_formatted')).show()

+-------------+
|std_formatted|
+-------------+
|       250.09|
+-------------+



In [80]:
df3.orderBy('Sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [87]:
df3.orderBy(col('Sales').desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



## Handling missing data

In [88]:
sc4 = SparkSession.builder.appName('Fix_Misses').getOrCreate()

In [90]:
df4 = sc4.read.csv('ContainsNull.csv',header=True,inferSchema=True)

In [91]:
df4.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [96]:
df4.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [97]:
df4.na.drop(thresh=2).show() #Giving threshold not null counts to consider it

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [98]:
df4.na.drop(how='all').show() #How to apply na Logic

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [99]:
df4.na.drop(subset=['Sales']).show() #na only on Sales Column

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [101]:
df4.na.fill("XXX",subset=["Name"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2|  XXX| null|
|emp3|  XXX|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [105]:
mean_sale = df4.select(mean('Sales')).collect()[0][0]
df4.na.fill(mean_sale,['Sales','Name']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



## Dates and Timestamps

In [5]:
spark = SparkSession.builder.appName("dates").getOrCreate()
df_dates = spark.read.csv("appl_stock.csv",header=True,inferSchema=True)

In [6]:
df_dates.head(1)

[Row(Date='2010-01-04', Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)]

In [7]:
df_dates.select('Date','Open').show(2)

+----------+----------+
|      Date|      Open|
+----------+----------+
|2010-01-04|213.429998|
|2010-01-05|214.599998|
+----------+----------+
only showing top 2 rows



In [8]:
from pyspark.sql.functions import *

In [10]:
df_dates.select(dayofmonth(df_dates['Date'])).show()

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
|              11|
|              12|
|              13|
|              14|
|              15|
|              19|
|              20|
|              21|
|              22|
|              25|
|              26|
|              27|
|              28|
|              29|
|               1|
+----------------+
only showing top 20 rows



In [12]:
df_dates.select(month(df_dates['Date'])).show(5)

+-----------+
|month(Date)|
+-----------+
|          1|
|          1|
|          1|
|          1|
|          1|
+-----------+
only showing top 5 rows



In [21]:
newdf = df_dates.withColumn('Year',year(df_dates['Date']))

In [23]:
# newdf.printSchema()
newdf2 = newdf.groupBy('Year').mean().select('Year','avg(Close)')


In [26]:
newdf2.select('Year',format_number('avg(Close)',2).alias('Average Closing Price')).orderBy(newdf2['Year'].desc()).show()

+----+---------------------+
|Year|Average Closing Price|
+----+---------------------+
|2016|               104.60|
|2015|               120.04|
|2014|               295.40|
|2013|               472.63|
|2012|               576.05|
|2011|               364.00|
|2010|               259.84|
+----+---------------------+

